In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')

data = pd.read_csv("example_products.csv")

# Combine features for recommendations
data['combined'] = data['product_name'] + ' ' + data['category'] + ' ' + data['description']

vectorizer = TfidfVectorizer(stop_words='english')
feature_vectors = vectorizer.fit_transform(data['combined'])

similarity = cosine_similarity(feature_vectors)

sia = SentimentIntensityAnalyzer()

def analyze_sentiment(review):
    score = sia.polarity_scores(review)
    compound = score['compound']
    if compound >= 0.05:
        return 'Positive'
    elif compound <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

data['sentiment'] = data['reviews'].apply(analyze_sentiment)

sentiment_map = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
data['sentiment_score'] = data['sentiment'].map(sentiment_map)

# ---------------------- RECOMMENDATION FUNCTION ----------------------
def recommend(product_name, num_recommendations=5):
    product_name = product_name.strip()
    
    if product_name not in data['product_name'].values:
        print(f"\n Product '{product_name}' not found in database.")
        print("Here are some available products you can try:")
        for name in data['product_name'].sample(5, random_state=1).values:
            print(f" - {name}")
        return
    
    idx = data[data['product_name'] == product_name].index[0]
    
    sim_scores = list(enumerate(similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_products = sim_scores[1:num_recommendations+1]
    
    print(f"\nTop {num_recommendations} Recommendations for '{product_name}':\n")
    for i, score in top_products:
        print(f" {data.iloc[i]['product_name']} | Category: {data.iloc[i]['category']} | Similarity: {score:.2f} | Sentiment: {data.iloc[i]['sentiment']}")

print(" Welcome to RecomMind  – Product Recommender + Sentiment Analyzer")
print("------------------------------------------------------------------")

while True:
    product = input("\nEnter a product name (or type 'exit' to quit): ")
    if product.lower() == 'exit':
        print("\n Thank you for using RecomMind! Goodbye! ")
        break

    recommend(product)
    
    # Also show sentiment summary for the selected product
    if product in data['product_name'].values:
        product_data = data[data['product_name'] == product]
        print(f"\n Sentiment Summary for '{product}':")
        print(f" - Review: {product_data.iloc[0]['reviews']}")
        print(f" - Sentiment: {product_data.iloc[0]['sentiment']}")